In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import cv2
from googleport cv2_ims.colab.patches imhow #Because cv2.imshow is no more supported by colab

In [ ]:
import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

In [ ]:
face_test = cv2.imread('/content/drive/MyDrive/face recognition project/face_test.png', cv2.IMREAD_UNCHANGED)
cv2_imshow(face_test)

# Generating dataset

In [ ]:
#Check this https://www.youtube.com/watch?v=LopYA64KmdE
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("/content/drive/MyDrive/face recognition project/haarcascade_frontalface_default.xml")
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        
        if faces == ():
            return None
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h,x:x+w]
        return cropped_face
    
    cap = cv2.VideoCapture("/content/drive/MyDrive/2_people_video.mp4")
    img_id = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id+=1
            face = cv2.resize(face_cropped(frame), (200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "/content/drive/MyDrive/data/"+"Ishwar."+str(img_id)+".jpg"
            #file_name_path = "/content/drive/MyDrive/Images for visualization/"+str(img_id)+'.jpg'
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2 )
            
            cv2_imshow(face)
            if cv2.waitKey(1)==13 or int(img_id)==20:
                break
                
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed !!!")
#generate_dataset()

In [ ]:
generate_dataset()

# Create label


In [ ]:
import numpy as np # pip install numpy

In [ ]:
def my_label(image_name):
    name = image_name.split('.')[-3] 
    # if you have two person in your dataset
    if name=="Ishwar":
        return np.array([1,0])
    elif name=="Manish":
        return np.array([0,1])
    
    
    # # if you have three person in your dataset
    # if name=="Ishwar":
    #     return np.array([1,0,0])
    # elif name=="Manish":
    #     return np.array([0,1,0])
    # elif name=="Bijay":
    #     return np.array([0,0,1])

# Create data

In [ ]:
import os
from random import shuffle
from tqdm import tqdm

In [ ]:
def my_data():
    data = []
    for i in range(1,21):
      img="/content/drive/MyDrive/face recognition project/data/"+"Ishwar."+str(i)+".jpg"
      img_data = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
      img_data.resize((50,50))
      data.append([np.array(img_data), my_label(img)])
    shuffle(data)  
    return data

In [ ]:
data = my_data()

In [ ]:
train = data[:15]  
test = data[5:]
X_train = np.array([i[0] for i in train]).reshape(-1,50,50,1)
print(X_train.shape)
y_train = [i[1] for i in train]
X_test = np.array([i[0] for i in test]).reshape(-1,50,50,1)
print(X_test.shape)
y_test = [i[1] for i in test]

(15, 50, 50, 1)
(15, 50, 50, 1)


In [ ]:
len(data)

20

# Creating the model 

In [ ]:
# tf.reset_default_graph()
convnet = input_data(shape=[50,50,1])
convnet = conv_2d(convnet, 32, 5, activation='relu')
# 32 filters and stride=5 so that the filter will move 5 pixel or unit at a time
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 3, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate = 0.001, loss='categorical_crossentropy')
model = tflearn.DNN(convnet, tensorboard_verbose=1)
model.fit(X_train, y_train, n_epoch=12, validation_set=(X_test, y_test), show_metric = True, run_id="FRS" )

IndexError: ignored

# Visualization & Prediction

In [ ]:
def data_for_visualization():
    Vdata = []
    for img in tqdm(os.listdir("/content/drive/MyDrive/face recognition project/Images for visualization")):
        path = os.path.join("/content/drive/MyDrive/face recognition project/Images for visualization", img)
        img_num = img.split('.')[0] 
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (50,50))
        Vdata.append([np.array(img_data), img_num])
    shuffle(Vdata)
    return Vdata

In [ ]:
Vdata = data_for_visualization()

In [ ]:
import matplotlib.pyplot as plt   # pip install matplotlib

fig = plt.figure(figsize=(20,20))
for num, data in enumerate(Vdata[:20]):
    img_data = data[0]
    y = fig.add_subplot(5,5, num+1)
    image = img_data
    data = img_data.reshape(50,50,1)
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 0:
        my_label = 'Ishwar'
    elif np.argmax(model_out) == 1:
        my_label = 'Manish'
    else:
        my_label = 'Bijay'
        
    y.imshow(image, cmap='gray')
    plt.title(my_label)
    
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()